# Windows Processes Suspicious Parent Directory
Detect suspicious parent processes of well-known Windows processes

## Rule Content
```
- title: Windows Processes Suspicious Parent Directory
  id: 96036718-71cc-4027-a538-d1587e0006a7
  status: experimental
  description: Detect suspicious parent processes of well-known Windows processes
  author: vburov
  references:
  - https://securitybytes.io/blue-team-fundamentals-part-two-windows-processes-759fe15965e2
  - https://www.carbonblack.com/2014/06/10/screenshot-demo-hunt-evil-faster-than-ever-with-carbon-black/
  - https://www.13cubed.com/downloads/windows_process_genealogy_v2.pdf
  - https://attack.mitre.org/techniques/T1036/
  date: 2019/02/23
  modified: 2019/08/20
  tags:
  - attack.defense_evasion
  - attack.t1036
  logsource:
    category: process_creation
    product: windows
    service: null
  detection:
    selection:
      Image:
      - '*\svchost.exe'
      - '*\taskhost.exe'
      - '*\lsm.exe'
      - '*\lsass.exe'
      - '*\services.exe'
      - '*\lsaiso.exe'
      - '*\csrss.exe'
      - '*\wininit.exe'
      - '*\winlogon.exe'
    filter:
      ParentImage:
      - '*\System32\\*'
      - '*\SysWOW64\\*'
      - '*\SavService.exe'
      - '*\Windows Defender\\*\MsMpEng.exe'
    filter_null:
      ParentImage: null
    condition: selection and not filter and not filter_null
  falsepositives:
  - Some security products seem to spawn these
  level: low

```

## Querying Elasticsearch

### Import Libraries

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd

### Initialize Elasticsearch client

In [ ]:
es = Elasticsearch(['http://helk-elasticsearch:9200'])
searchContext = Search(using=es, index='logs-*', doc_type='doc')

### Run Elasticsearch Query

In [ ]:
s = searchContext.query('query_string', query='((process_path.keyword:(*\\svchost.exe OR *\\taskhost.exe OR *\\lsm.exe OR *\\lsass.exe OR *\\services.exe OR *\\lsaiso.exe OR *\\csrss.exe OR *\\wininit.exe OR *\\winlogon.exe) AND (NOT (process_parent_path.keyword:(*\\System32\\* OR *\\SysWOW64\\* OR *\\SavService.exe OR *\\Windows\ Defender\\*\\MsMpEng.exe)))) AND (NOT (NOT _exists_:process_parent_path)))')
response = s.execute()
if response.success():
    df = pd.DataFrame((d.to_dict() for d in s.scan()))

### Show Results

In [ ]:
df.head()